# Descriptors

... or "ever wonder how `@property`, `@classmethod`, and `@staticmethod` work?"

**Descriptors** are object which contain one or more of the following magic methods, and which occur in a class body:

- `__get__(self, inst, cls)` - called when the descriptor attribute is looked up (e.g. `getattr()`)
- `__set__(self, inst, value)` - called when the descriptor attribute is set (e.g. `setattr()`)
- `__delete__(self, inst)` - called when the descriptor attribute is deleted (e.g. `delattr()` or `del inst.attr`)
- `__set_name__(self, cls, name)` - called to notify the descriptor of its name within the class

In [ ]:
class MyDesc:
    
    def __get__(self, inst, cls):
        # default behavior is something like return inst.__dict__[my_own_name]
        print(f'Calling __get__({self}, {inst}, {cls})')
        if inst is None:
            return self

    def __set__(self, inst, value):
        print(f'Calling __set__({self}, {inst}, {value})')

    def __delete__(self, inst):
        print(f'Calling __delete__({self}, {inst})')
        
    def __set_name__(self, cls, name):
        print(f'Setting name of the descriptor {self} in class {cls} to {name}')

In [ ]:
the_descriptor = MyDesc()

class MyClass:
    a = the_descriptor       # a.__set_name__ is called at class creation time
    b = the_descriptor
    c = the_descriptor
    d = MyDesc()
    
    def __repr__(self):
        return '<Instance of MyClass>'

```python
# Psuedo-code for what's happening at class creation time
for key, value in dct.items():
    if hasattr(value, '__set_name__'):
        value.__set_name__(key)
```

In [ ]:
myobj = MyClass()

In [ ]:
MyClass.a   # invokes a.__get__(None, MyClass)

In [ ]:
myobj.a     # invoke a.__get__(myobj, MyClass)

In [ ]:
# MyClass.a = 5  # Overwrites the descriptor, so don't do this

In [ ]:
myobj.a = 20   # invokes a.__set__(myobj, 20)

In [ ]:
del myobj.a    # invokes a.__delete__(myobj)

Let's re-implement `@property`:

In [ ]:
class myproperty:

    def __init__(self, getter, setter=None, deleter=None):
        self._getter = getter
        self._setter = setter
        self._deleter = deleter
        
    def __get__(self, inst, cls):
        print('Calling __get__')
        if inst is None:
            return self
        return self._getter(inst)
    
    def __set__(self, inst, value):
        print('Calling __set__')
        if self._setter is None:
            raise TypeError('value is read-only')
        self._setter(inst, value)
        
    def __delete__(self, inst):
        print('Calling __delete__')
        if self._deleter is None:
            raise TypeError('value is undeleteable')
        self._deleter(inst)
        
    def setter(self, setter):
        """Decorator to add a setter"""
        self._setter = setter
        return self
    
    def deleter(self, deleter):
        """Decorator to add a deleter"""
        self._deleter = deleter
        return self

In [ ]:
class Foo:   # Foo()    Foo(object)
    
    def get_bar(self):
        print('Calling getter for bar')
        return 'barval'
    
    def set_bar(self, value):
        print('Calling setter for bar')

    def del_bar(self):  # del foo.bar
        print('Calling deleter for bar')
        
    bar = myproperty(get_bar, set_bar, del_bar)

foo = Foo()

In [ ]:
class Foo:   # Foo()    Foo(object)
    
    @myproperty
    def bar(self):
        print('Calling getter for bar')
        return 'barval'
    #bar = myproperty(bar)
    
    @bar.setter
    def bar(self, value):
        print('Calling setter for bar')
    # bar is an instance of myproperty on this line
        
#     _tmp0 = bar.setter
#     def bar(self, value):
#         print('Calling the setter for bar')
#     bar = _tmp0(bar)
        
    @bar.deleter
    def bar(self):  # del foo.bar
        print('Calling deleter for bar')

foo = Foo()

In [ ]:
Foo.__dict__['bar']

In [ ]:
Foo.bar

In [ ]:
foo.bar

In [ ]:
foo.bar = 10

In [ ]:
foo.bar

In [ ]:
del foo.bar

In [ ]:
Foo.bar

## Descriptor types

- A **data descriptor** is a descriptor that defines both `__get__` and `__set__`
- A **non-data descriptor** is a descriptor that defines only `__get__`

> Data and non-data descriptors differ in how overrides are calculated with respect to entries in an instance’s dictionary. If an instance’s dictionary has an entry with the same name as a data descriptor, the data descriptor takes precedence. If an instance’s dictionary has an entry with the same name as a non-data descriptor, the dictionary entry takes precedence.

So order of precedence in attribute access is:

- data descriptor
- instance `__dict__`
- non-data descriptor

In [ ]:
class Foo(): 
    pass

foo = Foo()
foo.a = 5

In [ ]:
foo.__dict__

In [ ]:
class DataDescriptor:
    
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return 'data descriptor value'
    
    def __set__(self, obj, value):
        # Just make it a read-only data descriptor
        raise TypeError('read-only property')

In [ ]:
class NonDataDescriptor:
    
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return 'non-data descriptor value'
        

In [ ]:
class MyClass:
    data = DataDescriptor()
    nondata = NonDataDescriptor()

In [ ]:
obj = MyClass()
obj.__dict__.update(
    data='instance data',
    nondata='instance nondata'
)

Data descriptors have precedence over instance data:

In [ ]:
obj.data

Instance data has precendence over non-data descriptors:

In [ ]:
obj.nondata

In [ ]:
obj.nondata = 'something else'   # puts the data in the instance dict

In [ ]:
obj.nondata

If we delete it from the instance dict, however, it _will_ invoke the non-data descriptor's `__get__` method

In [ ]:
del obj.nondata

In [ ]:
obj.nondata

## Descriptor use case: cached property

In [ ]:
class cached_property:  # "reify" is another name for this
    
    def __init__(self, getter):
        self._getter = getter
        self._name = None
        
    def __set_name__(self, cls, name):
        self._name = name
    
    def __get__(self, inst, cls):
        if inst is None:
            return self
        value = self._getter(inst)
        setattr(inst, self._name, value) # put the value in the instance __dict__
        # alternatively, inst.__dict__[name] = value
        return value
    
#     def __set__(self, obj, value):
#         obj.__dict__[self._name] = value

In [ ]:
class CachedExample:
    
    @cached_property
    def prop(self):
        print('Calculating CachedExample.prop')
        return 42


In [ ]:
ce = CachedExample()

In [ ]:
ce.prop # adds prop to ce.__dict__

In [ ]:
ce.prop  # Since prop is non-data descriptor, it is not even accessed here

In [ ]:
import sys
sys.version_info

In "real life", this is implemented in `functools.cached_property` (in Python 3.8+)

```python
import functools
help(functools.cached_property)
```

In [ ]:
import functools
help(functools.cached_property)

In [ ]:
functools.cached_property??

## (surprising) use case: plain vanilla methods

(where does the `self` come from?)

Let's implement a "method" (using the `__call__` magic method)

In [ ]:
class BoundMethod:
    def __init__(self, inst, function):
        self._inst = inst
        self._function = function
        
    def __repr__(self):
        return f'<Bound method {self._inst!r}.{self._function!r}'
    
    def __call__(self, *args, **kwargs):
        return self._function(self._inst, *args, **kwargs)
        
        
class UnboundMethod:
    def __init__(self, function):
        self._function = function
        
    def __repr__(self):
        return f'<Unbound method {self._function}'

    def __call__(self, *args, **kwargs):
        return self._function(*args, **kwargs)
    
    def __get__(self, inst, cls):
        print(f"Calling UnboundMethod.__get__({inst}, {cls})")
        if inst is None:
            return self._function
        else:
            return BoundMethod(inst, self._function)

In [ ]:
def amethod_function(self):
    print(f'Calling amethod_function with self={self!r}')

In [ ]:
class MyVeryStrangeClass:
    def __init__(self, name):
        self._name = name
        
    def __repr__(self):
        return f'<VeryStrange {self._name!r}>'
        
    amethod = UnboundMethod(amethod_function)
    
    def regular_method(self):
        print('Calling a regular method with', self)

In [ ]:
MyVeryStrangeClass.amethod

In [ ]:
MyVeryStrangeClass.regular_method

In [ ]:
foo = MyVeryStrangeClass('foo')
foo.amethod

In [ ]:
foo.regular_method

In [ ]:
foo.amethod()

In [ ]:
foo.regular_method()

**Surely** that can't be how it's implemented.... that would mean that Python functions actually have a `__get__` method....

In [ ]:
amethod_function.__get__

oh no...

In [ ]:
amethod = amethod_function.__get__(foo, MyVeryStrangeClass)

In [ ]:
amethod

In [ ]:
amethod()

Open [Descriptors Lab](./descriptors-lab.ipynb)